## 装饰器

In [1]:
def fn():
    '''this is fn'''

In [2]:
help(fn)

Help on function fn in module __main__:

fn()
    this is fn



In [4]:
fn.__doc__ # 存储函数的文档

'this is fn'

In [8]:
fn.__name__ # 存储函数的名字,

# dir（fn） 查看所有的fn 内嵌方法

'fn'

In [31]:
def logger(fn):
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('{} called took {}'.format(fn.__name__, end - start))
        return ret
    copy_propertities(fn, wrap)
    return wrap

In [32]:
def copy_propertities(src, dst):
    dst.__name__ = src.__name__
    dst.__doc__ = src.__doc__

In [33]:
@logger
def sleep(x):
    '''this is sleep'''
    time.sleep(x)

In [34]:
import time, datetime

In [35]:
sleep.__doc__

'this is sleep'

In [36]:
sleep.__name__

'sleep'

In [38]:
def copy_propertities(src): # 柯里化
    def _copy(dst):
        dst.__name__ = src.__name__
        dst.__doc__ = src.__doc__
    return _copy

In [39]:
def logger(fn):
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('{} called took {}'.format(fn.__name__, end - start))
        return ret
    copy_propertities(fn)(wrap)
    return wrap

In [40]:
@logger
def sleep(x):
    '''this is sleep'''
    time.sleep(x)

In [41]:
sleep.__doc__

'this is sleep'

In [42]:
sleep.__name__

'sleep'

In [43]:
def copy_propertities(src): # 柯里化，带参数的装饰器
    def _copy(dst):
        dst.__name__ = src.__name__
        dst.__doc__ = src.__doc__
        return dst
    return _copy

In [44]:
def logger(fn):
    @copy_propertities(fn)
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('{} called took {}'.format(fn.__name__, end - start))
        return ret
    return wrap

In [45]:
@logger
def sleep(x):
    '''this is sleep'''
    time.sleep(x)

In [46]:
sleep.__doc__

'this is sleep'

In [47]:
sleep.__name__

'sleep'

AOP: 面向切面编程，为了解决一类问题

In [48]:
import functools

In [49]:
help(functools.wraps)

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



In [50]:
def logger(fn):
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        start = datetime.datetime.now()
        ret = fn(*args, **kwargs)
        end = datetime.datetime.now()
        print('{} called took {}'.format(fn.__name__, end - start))
        return ret
    return wrap

In [51]:
@logger(2)
def sleep(x):
    '''this is sleep'''
    time.sleep(x)

In [52]:
sleep.__name__

'sleep'

In [54]:
sleep.__doc__

'this is sleep'

In [55]:
start = datetime.datetime.now()

In [56]:
end = datetime.datetime.now()

In [59]:
delta = end - start

In [66]:
delta.seconds

3

In [67]:
def logger(s):
    def _logger(fn):
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            start = datetime.datetime.now()
            ret = fn(*args, **kwargs)
            end = datetime.datetime.now()
            if (end - start).seconds > s:
                print('{} called took {}'.format(fn.__name__, end - start))
            return ret
        return wrap
    return _logger

In [68]:
@logger(2)
def sleep(x):
    time.sleep(x)

In [69]:
sleep(3)

sleep called took 0:00:03.000528


In [70]:
sleep(1)

In [71]:
logger(2)(sleep)(3)

sleep called took 0:00:03.002246
sleep called took 0:00:03.002534


In [76]:
def logger(s, p=lambda name, t: print('call {} took {}'.format(name, t))):
    def _logger(fn):
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            start = datetime.datetime.now()
            ret = fn(*args, **kwargs)
            end = datetime.datetime.now()
            if (end - start).seconds > s:
                p(fn.__name__, end - start)
            return ret
        return wrap
    return _logger

In [77]:
@logger(2)
def sleep(x):
    time.sleep(x)

In [78]:
sleep(1)

In [79]:
sleep(3)

call sleep took 0:00:03.002283


所以，装饰器的作用就是 不改变函数本身的情况下，做一些预定义的逻辑处理

In [80]:
@logger(2, p=lambda name, t: None)
def sleep(x):
    time.sleep(x)

In [81]:
sleep(1)

In [82]:
sleep(3)

In [84]:
def logger(s):
    def _logger(p=lambda name, t: print('call {} took {}'.format(name, t))):
        def __logger(fn):
            @functools.wraps(fn)
            def wrap(*args, **kwargs):
                start = datetime.datetime.now()
                ret = fn(*args, **kwargs)
                end = datetime.datetime.now()
                if (end - start).seconds > s:
                    p(fn.__name__, end - start)
                return ret
            return wrap
        return __logger
    return _logger

In [91]:
@logger(2)()
def sleep(x):
    time.sleep(x)

# logger(2)()(sleep)(3)

SyntaxError: invalid syntax (<ipython-input-91-37e9c6a18c3f>, line 1)

In [86]:
logger(2)()(sleep)(3)

call sleep took 0:00:03.004451


In [87]:
lg = logger(2)

In [88]:
lg

<function __main__.logger.<locals>._logger>

In [89]:
@lg()
def sleep(x):
    time.sleep(x)

In [90]:
sleep(3)

call sleep took 0:00:03.000811


## 类型注解

In [92]:
a = 1

In [93]:
type(a)

int

In [94]:
a = 's'

In [95]:
def add(x, y):
    return x + y

In [96]:
add(1, 3)

4

In [97]:
add('a', 'b')

'ab'

In [98]:
add(1, 'a')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [102]:
def fn():
    '''
    总是返回True
    :return: True
    '''
    return False

In [104]:
def add(x: int, y: int) -> int:
    return x + y

- 冒号后面跟参数类型
- 箭头后面跟返回值类型

In [106]:
add.__annotations__

{'return': int, 'x': int, 'y': int}

In [107]:
import typing

In [108]:
def _sum(lst: typing.List[int]) -> int:
    ret = 0
    for x in lst:
        ret += x
    return ret

In [109]:
_sum.__annotations__

{'lst': typing.List[int], 'return': int}

In [110]:
import inspect

In [111]:
sig = inspect.signature(add)

In [121]:
list(sig.parameters.values())[0].annotation

int

In [117]:
sig.parameters['x'].annotation

int

In [138]:
sig

<Signature (x:int, y:int) -> int>

In [139]:
sig.parameters

mappingproxy({'x': <Parameter "x:int">, 'y': <Parameter "y:int">})

In [140]:
sig.parameters['x']

<Parameter "x:int">

In [142]:
sig.parameters['x'].annotation

int

In [143]:
sig.parameters.values()

odict_values([<Parameter "x:int">, <Parameter "y:int">])

In [144]:
sig.parameters['x'].name

'x'

In [122]:
param.name

'x'

In [116]:
param.annotation

int

In [ ]:
@type
def add(x: int, y: int) -> int:
    return x + y

add(1, 'a') # raise ,print

In [130]:
def typed(fn):
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        params = inspect.signature(fn).parameters
        
        # kwargs 检查
        for k, v in kwargs.items():
            if not isinstance(v, params[k].annotation):
                raise TypeError('parameter {} require {}, but {}'.format(k, params[k].annotation, type(k)))
                
        # args
        for i, arg in enumerate(args):
            param = list(params.values())[i]
            if not isinstance(arg, param.annotation):
                raise TypeError('parameter {} require {}, but {}'.format(param.name, param.annotation, type(arg)))

        return fn(*args, **kwargs)
    return wrap

In [127]:
@typed
def add(x: int, y: int) -> int:
    return x + y

In [129]:
add(1, 'a')

TypeError: parameter y require <class 'int'>, but <class 'str'>

In [131]:
add(y=3, x='a')

TypeError: parameter x require <class 'int'>, but <class 'str'>

In [132]:
@typed
def add(x: int, y) -> int:
    return x + y

In [133]:
add(1, 1)

TypeError: parameter y require <class 'inspect._empty'>, but <class 'int'>

In [134]:
def typed(fn):
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        params = inspect.signature(fn).parameters
        
        # kwargs 检查
        for k, v in kwargs.items():
            param = params[k]
            if param.annotation != inspect._empty and not isinstance(v, param.annotation):
                raise TypeError('parameter {} require {}, but {}'.format(k, params[k].annotation, type(k)))
                
        # args
        for i, arg in enumerate(args):
            param = list(params.values())[i]
            if param.annotation != inspect._empty and not isinstance(arg, param.annotation):
                raise TypeError('parameter {} require {}, but {}'.format(param.name, param.annotation, type(arg)))

        return fn(*args, **kwargs)
    return wrap

In [135]:
@typed
def add(x: int, y) -> int:
    return x + y

In [137]:
add(1, 'a')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [145]:
def add(x, y):
    return x + y

In [146]:
add(3, 5)

8

In [147]:
new_add = functools.partial(add, y=3)

In [148]:
new_add(1)

4

In [150]:
new_add(2, y=5)

7

**接受一个函数参数 然后把这个函数的某些参数变成默认参数 并返回一个新函数**

In [153]:
def cmp(a, b, r):
    if r:
        return a < b
    return a > b

In [154]:
def sort(it, cmp=lambda a, b: a < b):
    ret = []
        
    for x in it:
        for i, e in enumerate(ret):
            if cmp(x, e):
                ret.insert(i, x)
                break
            
        else:
            ret.append(x)
    return ret

In [162]:
sort(range(4), functools.partial(cmp, r=True))

[0, 1, 2, 3]

In [163]:
def logger(s):
    def _logger(fn):
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            start = datetime.datetime.now()
            ret = fn(*args, **kwargs)
            end = datetime.datetime.now()
            if (end - start).seconds > s:
                print('{} called took {}'.format(fn.__name__, end - start))
            return ret
        return wrap
    return _logger

In [169]:
@logger(2)
@functools.lru_cache(2)
def sleep(x):
    time.sleep(x)
    

In [176]:
sleep(3)

sleep called took 0:00:03.005083


In [171]:
sleep(3)

In [172]:
sleep(4)

sleep called took 0:00:04.001363


In [173]:
sleep(4)

In [174]:
sleep(5)

sleep called took 0:00:05.001122


In [177]:
sleep(5)

- 不需要过期
- 不需要清除
- 不需要分布式
- 函数必须是无副作用的

- 可过期
- 可清除
- 不换出